# ALMA-IMF+SPICY tool: Filter Convolution
Workspace to convolve SED filters into a robitaille_models folder. Search for "!filepath!" to locate files/filepath references if they need to be changed.

In [ ]:
import numpy as np
from sedfitter.filter import Filter
from sedfitter.convolve import convolve_model_dir
from astropy import units as u
from astroquery.svo_fps import SvoFps
from glob import glob
import os

In [ ]:
#define ALMA bands 3 and 6
almaimf_bandends_1mm = [[216.10085679, 216.36181569],
                        [217.05104378, 217.31175857],
                        [219.90488464, 220.04866835],
                        [218.13102322, 218.39222624],
                        [219.51976276, 219.66379059],
                        [230.31532951, 230.81137113],
                        [231.06503709, 231.56181105],
                        [231.52507012, 233.42623749]]*u.GHz

nu_1mm = np.linspace(almaimf_bandends_1mm.min(), almaimf_bandends_1mm.max(), 5000)
response_1mm = np.zeros(nu_1mm.size, dtype='bool')
for start, stop in almaimf_bandends_1mm:
    response_1mm |= (nu_1mm > start) & (nu_1mm < stop)

almaimf_bandends_3mm = [[ 93.13410936,  93.25141259],
                        [ 91.75059068,  92.68755174],
                        [102.15273354, 103.0896946 ],
                        [104.55323851, 105.49019957]]*u.GHz

nu_3mm = np.linspace(almaimf_bandends_3mm.min(), almaimf_bandends_3mm.max(), 5000)
response_3mm = np.zeros(nu_3mm.size, dtype='bool')

for start, stop in almaimf_bandends_3mm:
    response_3mm |= (nu_3mm > start) & (nu_3mm < stop)

def convolve(filternames, dirs, includeALMA=True):
    filtercurves = {filtername: SvoFps.get_transmission_data(filtername) for filtername in filternames}
    wavelengths = [np.average(filtercurves[filtername]['Wavelength'],weights=filtercurves[filtername]['Transmission']) 
                   for filtername in filternames]

    filterfreqs = {filtername: u.Quantity(filtercurves[filtername]['Wavelength'], u.AA).to(u.Hz, u.spectral()) for filtername in filternames}
    filtertrans = {filtername: np.array(filtercurves[filtername]['Transmission'])[np.argsort(filterfreqs[filtername])]
                   for filtername in filternames}
    filterfreqs = {filtername: np.sort(filterfreqs[filtername]) for filtername in filternames}

    sed_filters = [Filter(name=filtername,central_wavelength=wl*u.AA,
                      nu=filterfreqs[filtername],response=filtertrans[filtername])
                   for filtername, wl in zip(filternames, wavelengths)]
                                                                        
    if includeALMA:
        sed_filters.append(Filter(name='user_filters/ALMA-IMF_1mm',central_wavelength=(228.15802*u.GHz).to(u.mm, u.spectral()),
                                  nu=nu_1mm,response=response_1mm.astype(float),))
        sed_filters.append(Filter(name='user_filters/ALMA-IMF_3mm',central_wavelength=(99.68314596*u.GHz).to(u.mm, u.spectral()),
                                  nu=nu_3mm,response=response_3mm.astype(float),))

    for filterfunc in sed_filters:
        filterfunc.normalize()

    for model_dir in dirs:
        convolve_model_dir(model_dir, sed_filters,overwrite=True)

In [ ]:
#copy theo's directory to mine
import shutil
#shutil.rmtree('/blue/adamginsburg/adamginsburg/ALMA_IMF/SPICY_ALMAIMF/BriceTingle/robitaille_models-1.2/')
shutil.copytree('/blue/adamginsburg/richardson.t/research/flux/robitaille_models-1.2/','/blue/adamginsburg/adamginsburg/ALMA_IMF/SPICY_ALMAIMF/BriceTingle/robitaille_models-1.2/') # !filepath!

In [ ]:
dirs_tingle = glob('/blue/adamginsburg/adamginsburg/ALMA_IMF/SPICY_ALMAIMF/BriceTingle/robitaille_models-1.2/s*') # !filepath!

In [ ]:
#make any missing directories
for model_dir in dirs_tingle:
    if not os.path.exists(f'{model_dir}/convolved'):
        os.makedirs(f'{model_dir}/convolved')
    if not os.path.exists(f'{model_dir}/convolved/user_filters'):
        os.makedirs(f'{model_dir}/convolved/user_filters') # !filepath!

In [ ]:
#retrieve and convolve R17 filters + other specific ones
svo_filters = ['UKIRT/UKIDSS.J', 'UKIRT/UKIDSS.H', 'UKIRT/UKIDSS.K',
               '2MASS/2MASS.J','2MASS/2MASS.H','2MASS/2MASS.Ks','Herschel/Pacs.blue',
               'Herschel/Pacs.green','Herschel/Pacs.red','Herschel/SPIRE.PSW','Herschel/SPIRE.PMW',
               'Herschel/SPIRE.PLW','Paranal/VISTA.Y','Paranal/VISTA.Z','Paranal/VISTA.J',
               'Paranal/VISTA.H','Paranal/VISTA.Ks','Spitzer/IRAC.I1','Spitzer/IRAC.I2',
               'Spitzer/IRAC.I3','Spitzer/IRAC.I4','Spitzer/MIPS.24mu','Spitzer/MIPS.70mu',
               'Spitzer/MIPS.160mu','WISE/WISE.W1','WISE/WISE.W2','WISE/WISE.W3','WISE/WISE.W4']
for model_dir in dirs_tingle:
    for name in svo_filters:
        instrument = name.split('/')[0]
        if not os.path.exists(f'{model_dir}/convolved/{instrument}'):
            os.system(f'mkdir {model_dir}/convolved/{instrument}') # !filepath!
convolve(svo_filters,dirs_tingle,includeALMA=True)

In [ ]:
#retrieve and convolve JWST filters
jwst_filters = SvoFps.get_filter_list('JWST')['filterID']
for model_dir in dirs_tingle:
    for name in jwst_filters:
        instrument = name.split('/')[0]
        if not os.path.exists(f'{model_dir}/convolved/{instrument}'):
            os.system(f'mkdir {model_dir}/convolved/{instrument}') # !filepath!
convolve(jwst_filters,dirs_tingle,includeALMA=False)